In [2]:
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle, seed

from echo_state_network import ESN

Equation for the first pendulum: $L\ddot{\theta_{1}} = -gsin(\theta_{1}) - kL(sin(\theta_{1}) - sin(\theta_{2}))$

Equation for the second pendulum: $L\ddot{\theta_{1}} = -gsin(\theta_{2}) + kL(sin(\theta_{1}) - sin(\theta_{2}))$

# Equation independent of the second angle to get started

In [ ]:
def coupled_pendulum(u, t):
    #Should return y', y''
    # angle'' = -g/l * sin(angle) - damping * radius / mass * angle'
    
    return (u[1], -9.81 * np.sin(u[0]) - 4 * 0.8 / 10 * u[1]):
    